<a href="https://colab.research.google.com/github/Alisha210302/Deep-Learning/blob/main/HyperParameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
!pip install tensorflow

In [121]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [122]:
import numpy as np
import pandas as pd

In [123]:
df = pd.read_csv('diabetes.csv')

In [124]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [125]:
df.corr()['Outcome']

,Outcome
Pregnancies,0.221898
Glucose,0.466581
BloodPressure,0.065068
SkinThickness,0.074752
Insulin,0.130548
BMI,0.292695
DiabetesPedigreeFunction,0.173844
Age,0.238356
Outcome,1.000000


In [126]:
X = df.iloc[:,:-1].values
y = df.iloc[:,-1]

In [127]:
from sklearn.preprocessing import StandardScaler

In [128]:
scaler = StandardScaler()

In [129]:
X = scaler.fit_transform(X)

In [130]:
X.shape

(768, 8)

In [131]:
from sklearn.model_selection import train_test_split

In [132]:
X_train , X_test , y_train , y_test = train_test_split(X,y,test_size=0.2,random_state=1)

In [169]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense,Dropout

In [134]:
model = Sequential()
model.add(Dense(32,activation='relu',input_dim=8))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [135]:
model.fit(x=X_train,y=y_train,batch_size=32,epochs=100,validation_data=(X_test,y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.5174 - loss: 0.6865 - val_accuracy: 0.6364 - val_loss: 0.6376
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6563 - loss: 0.6355 - val_accuracy: 0.7143 - val_loss: 0.5867
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7301 - loss: 0.5822 - val_accuracy: 0.7922 - val_loss: 0.5506
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7366 - loss: 0.5491 - val_accuracy: 0.8117 - val_loss: 0.5260
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7696 - loss: 0.5296 - val_accuracy: 0.7987 - val_loss: 0.5088
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.7917 - loss: 0.4966 - val_accuracy: 0.7987 - val_loss: 0.4947
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.7696 - loss: 0.5037 - val_accuracy: 0.7857 - val_loss: 0.4865
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.7524 - loss: 0.5057 - val_accuracy: 0.7922 

In [136]:
# 1. How to select appropriate optimizer
# 2. No. of nodes in a layer
# 3. How to select no of layers
# 4. All in one model

# 1. How to select appropriate optimizer

In [137]:
!pip install -U keras-tuner

In [138]:
import keras_tuner as kt

In [139]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(32,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  optimizer=hp.Choice('optimizer',values=['adam','sgd','rmsprop','adadelta'])

  model.compile(optimizer=optimizer,loss='binary_crossentropy',metrics=['accuracy'])

  return model



In [140]:
tuner = kt.RandomSearch(build_model,
                     objective='val_accuracy',
                     max_trials=5)

Reloading Tuner from ./untitled_project/tuner0.json


In [141]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [142]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'rmsprop'}

In [143]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 6 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [144]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 32)                  │             288 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [145]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - accuracy: 0.7635 - loss: 0.5365 - val_accuracy: 0.7987 - val_loss: 0.5304
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.7593 - loss: 0.5257 - val_accuracy: 0.7987 - val_loss: 0.5169
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7337 - loss: 0.5282 - val_accuracy: 0.7987 - val_loss: 0.5064
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7758 - loss: 0.4967 - val_accuracy: 0.7857 - val_loss: 0.4989
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7745 - loss: 0.4790 - val_accuracy: 0.7857 - val_loss: 0.4932
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7730 - loss: 0.4793 - val_accuracy: 0.7857 - val_loss: 0.4891
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7769 - loss: 0.4745 - val_accuracy: 0.7857 - val_loss: 0.4861
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7697 - loss: 0.4774 - val_accuracy: 0.7

# 2. No. of nodes in a layer

In [146]:
def build_model(hp):

  model = Sequential()

  units = hp.Int('units',min_value=8,max_value=128,step=8)

  model.add(Dense(units=units,activation='relu',input_dim=8))
  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [147]:
tuner = kt.RandomSearch(build_model,
                     objective='val_accuracy',
                        directory='mydir',
                        project_name='myproject',
                     max_trials=5)

Reloading Tuner from mydir/myproject/tuner0.json


In [148]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [149]:
tuner.get_best_hyperparameters()[0].values

{'units': 128}

In [150]:
model = tuner.get_best_models(num_models=1)[0]

In [151]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 128)                 │           1,152 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             129 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,281 (5.00 KB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 0 (0.00 B)

In [152]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.7715 - loss: 0.4908 - val_accuracy: 0.7922 - val_loss: 0.4722
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7635 - loss: 0.4728 - val_accuracy: 0.8052 - val_loss: 0.4667
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7836 - loss: 0.4546 - val_accuracy: 0.7987 - val_loss: 0.4673
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7488 - loss: 0.4882 - val_accuracy: 0.8117 - val_loss: 0.4654
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7807 - loss: 0.4520 - val_accuracy: 0.8052 - val_loss: 0.4647
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8032 - loss: 0.4175 - val_accuracy: 0.7857 - val_loss: 0.4644
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7714 - loss: 0.4550 - val_accuracy: 0.7857 - val_loss: 0.4656
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7789 - loss: 0.4353 - val_accuracy: 0.77

# 3. How to select no of layers

In [154]:
def build_model(hp):

  model = Sequential()

  model.add(Dense(72,activation='relu',input_dim=8))

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    model.add(Dense(72,activation='relu'))

  model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer='rmsprop',loss='binary_crossentropy',metrics=['accuracy'])

  return model




In [155]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='num_layers')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [156]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

Trial 3 Complete [00h 00m 04s]
val_accuracy: 0.798701286315918

Best val_accuracy So Far: 0.8116883039474487
Total elapsed time: 00h 00m 11s


In [157]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [159]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [160]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))


Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.7743 - loss: 0.4977 - val_accuracy: 0.8182 - val_loss: 0.4689
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7816 - loss: 0.4743 - val_accuracy: 0.8052 - val_loss: 0.4586
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7725 - loss: 0.4608 - val_accuracy: 0.8247 - val_loss: 0.4558
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8016 - loss: 0.4338 - val_accuracy: 0.7727 - val_loss: 0.4641
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7813 - loss: 0.4228 - val_accuracy: 0.7922 - val_loss: 0.4593
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7709 - loss: 0.4422 - val_accuracy: 0.7727 - val_loss: 0.4768
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 26ms/step - accuracy: 0.7750 - loss: 0.4776 - val_accuracy: 0.7922 - val_loss: 0.4588
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.7793 - loss: 0.4544 - val_accuracy: 0.8

# 4. All in one model

In [170]:
def build_model(hp):

  model = Sequential()

  counter = 0

  for i in range(hp.Int('num_layers',min_value=1,max_value=10)):

    if counter == 0:

      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid']),
              input_dim=8
          )

      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))

    else:
      model.add(
          Dense(
              hp.Int('units'+str(i),min_value=8,max_value=128,step=8),
              activation=hp.Choice('activation'+str(i),values=['relu','tanh','sigmoid'])
          )
      )
      model.add(Dropout(hp.Choice('dropout'+str(i),values=[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])))
    counter+=1

    model.add(Dense(1,activation='sigmoid'))

  model.compile(optimizer=hp.Choice('optimizer',values=['nadam','adam','sgd','rmsprop','adadelta']),loss='binary_crossentropy',metrics=['accuracy'])

  return model

In [171]:
tuner = kt.RandomSearch(build_model,objective='val_accuracy',max_trials=3,directory='mydir',project_name='final')

Reloading Tuner from mydir/final/tuner0.json


In [172]:
tuner.search(X_train,y_train,epochs=5,validation_data=(X_test,y_test))

In [173]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2,
 'units0': 56,
 'activation0': 'sigmoid',
 'optimizer': 'nadam',
 'units1': 8,
 'activation1': 'relu'}

In [174]:
model = tuner.get_best_models(num_models=1)[0]

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'nadam', because it has 2 variables whereas the saved optimizer has 19 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [175]:
model.fit(X_train,y_train,batch_size=32,epochs=100,initial_epoch=6,validation_data=(X_test,y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 3s 21ms/step - accuracy: 0.6375 - loss: 0.6796 - val_accuracy: 0.6494 - val_loss: 0.6721
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6677 - loss: 0.6702 - val_accuracy: 0.6429 - val_loss: 0.6661
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6832 - loss: 0.6616 - val_accuracy: 0.6429 - val_loss: 0.6613
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6680 - loss: 0.6609 - val_accuracy: 0.6429 - val_loss: 0.6572
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6413 - loss: 0.6615 - val_accuracy: 0.6429 - val_loss: 0.6529
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.6526 - loss: 0.6538 - val_accuracy: 0.6429 - val_loss: 0.6482
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6754 - loss: 0.6467 - val_accuracy: 0.6494 - val_loss: 0.6435
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.6449 - loss: 0.6503 - val_accuracy: 0.66